# **Learn Prompt Engineering — Intern Exercise Workbook**

This workbook covers core prompt engineering techniques: summarization, question answering, classification, information extraction, translation, tone transformation, role playing, code generation, reasoning, few-shot prompting, and chain-of-thought (CoT).

---

### 📋 Instructions
- Each section has **questions** with `# TODO` or `___` blanks.
- **Fill in the blanks** and **run each cell** to verify.
- 🎯 = Hint | 🏆 = Bonus Challenge

---

## Section 1: Setup

We need to install libraries and create a helper function to talk to the LLM.

### Q1.1 — Install Required Libraries

**Task:** Install `openai` (version 0.28) and `rich` (for pretty printing).

In [ ]:
# TODO: Install openai==0.28 and rich

!pip install ___
!pip install ___

### Q1.2 — Create the OpenAI Client

**Task:** Set up the OpenAI client using the newer `openai.OpenAI()` approach.

🎯 **Hint:** The client is created with `openai.OpenAI(api_key="...")`

In [ ]:
import os
import openai
from rich import print

# TODO: Create the OpenAI client with your API key
client = openai.OpenAI(
    api_key= "___"  # Replace with your actual API key
)

### Q1.3 — Understand the Chat API Structure

The Chat API uses **messages** with different **roles**:
- `system` — Sets the AI's behavior/persona
- `user` — The human's message
- `assistant` — The AI's previous response (for multi-turn)

**Task:** Fill in the missing roles and content in the API call below. The conversation should:
1. Set a **system** message: "You are an AI research assistant. You use a tone that is technical and scientific."
2. Have a **user** greeting: "Hello, who are you?"
3. Have an **assistant** reply: "Greeting! I am an AI research assistant. How can I help you today?"
4. Have a **user** follow-up: "Can you tell me about the creation of black holes?"

In [ ]:
MODEL = "gpt-3.5-turbo"

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "___", "content": "___"},  # System persona
        {"role": "___", "content": "___"},  # User greeting
        {"role": "___", "content": "___"},  # Assistant reply
        {"role": "___", "content": "___"}   # User follow-up question
    ],
    temperature=0,
)

# TODO: Print only the text content from the response
# 🎯 Hint: response.choices[0].message.content
print(___)

### Q1.4 — Build a Reusable Helper Function

**Task:** Complete the `get_completion` function that:
- Accepts `messages` (list of dicts), `model`, `temperature`, and `max_tokens`
- Calls the Chat Completions API
- Returns **only the text content**

In [ ]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=4000):
    """Send messages to the Chat API and return the response text."""

    # TODO: Call client.chat.completions.create() with the 4 parameters
    response = client.chat.completions.create(
        model=___,
        messages=___,
        temperature=___,
        max_tokens=___,
    )

    # TODO: Return the text content
    return ___

---

## Section 2: Basic Prompt & Text Completion

The simplest form of prompting — give the model a partial sentence and let it complete.

### Q2.1 — Text Completion

**Task:** Send the prompt `"The sky is"` to the model and print the completion. Wrap it in the correct message format.

In [ ]:
prompt = "The sky is"

# TODO: Create the message list with the correct role and content
message = [
    {
        "role": "___",
        "content": ___
    }
]

response = get_completion(message)
print(response)

---

## Section 3: Text Summarization

Summarization condenses long text into short, digestible content. We'll see how **iterative refinement** improves results.

### Q3.1 — Basic Summarization (Append Instruction)

**Task:** Given the paragraph about antibiotics below, add an instruction at the end to summarize it in one sentence.

🎯 **Hint:** Simply append `"Explain the above in one sentence:"` after the text.

In [ ]:
# TODO: Complete the prompt — add the text and the summarization instruction

prompt = """Antibiotics are a type of medication used to treat bacterial infections. They work by either killing the bacteria or preventing them from reproducing, allowing the body's immune system to fight off the infection. Antibiotics are usually taken orally in the form of pills, capsules, or liquid solutions, or sometimes administered intravenously. They are not effective against viral infections, and using them inappropriately can lead to antibiotic resistance.

___"""

message = [{"role": "user", "content": prompt}]

response = get_completion(message, temperature=0)
print(response)

### Q3.2 — Better Summarization (Task-First Approach)

A better pattern: **state the task FIRST**, then provide the data.

**Task:** Rewrite the prompt so the instruction comes before the abstract. Use the label `Abstract:` to separate them.

In [ ]:
# TODO: Write the prompt with instruction FIRST, then the abstract

prompt = """
___  # Instruction: what should the model do?

Abstract: Antibiotics are a type of medication used to treat bacterial infections. They work by either killing the bacteria or preventing them from reproducing, allowing the body's immune system to fight off the infection. Antibiotics are usually taken orally in the form of pills, capsules, or liquid solutions, or sometimes administered intravenously. They are not effective against viral infections, and using them inappropriately can lead to antibiotic resistance.
"""

message = [{"role": "user", "content": prompt}]

response = get_completion(message, temperature=0)
print(response)

### Q3.3 — Iterative Refinement (Adding Constraints)

**Task:** Further improve the prompt by adding a **constraint**: `"Avoid technical jargon and explain it in the simplest of words."`

🏆 **Compare:** How does the output differ from Q3.2?

In [ ]:
# TODO: Add the simplification constraint to the prompt

prompt = """
Your task is to summarize an abstract into one sentence.

___  # Add the simplification constraint here

Abstract: Antibiotics are a type of medication used to treat bacterial infections. They work by either killing the bacteria or preventing them from reproducing, allowing the body's immune system to fight off the infection. Antibiotics are usually taken orally in the form of pills, capsules, or liquid solutions, or sometimes administered intravenously. They are not effective against viral infections, and using them inappropriately can lead to antibiotic resistance.
"""

message = [{"role": "user", "content": prompt}]

response = get_completion(message, temperature=0)
print(response)

---

## Section 4: Question Answering

LLMs can answer questions based on a provided **context**. The key is to clearly separate the context, question, and expected behavior.

### Q4.1 — Context-Based QA

**Task:** Complete the prompt below to:
1. Instruct the model to answer based **only** on the context
2. Keep the answer short and concise
3. Respond `"Unsure about answer"` if not confident
4. Ask: "What was OKT3 originally sourced from?"

In [ ]:
# TODO: Fill in the instruction, fallback behavior, and question

prompt = """___  # Instruction: answer based on context, keep short, respond "Unsure" if not sure

Context: Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. There, scientists generated an early version of the antibody, dubbed OKT3. Originally sourced from mice, the molecule was able to bind to the surface of T cells and limit their cell-killing potential. In 1986, it was approved to help prevent organ rejection after kidney transplants, making it the first therapeutic antibody allowed for human use.

Question: ___

Answer:
"""

message = [{"role": "user", "content": prompt}]

response = get_completion(message)
print(response)

---

## Section 5: Text Classification

LLMs can classify text into categories — sentiment analysis, topic classification, etc.

### Q5.1 — Sentiment Classification

**Task:** Complete the prompt to classify user input into `neutral`, `negative`, or `positive`. Use triple backticks ` ``` ` as delimiters around the input text.

🎯 **Hint:** Use `{user_input}` inside the backticks and `.format()` to inject the value.

In [ ]:
user_input = "I think the food was okay"

# TODO: Write the classification prompt with delimited input
prompt = """___  # Instruction: classify into neutral/negative/positive

Text: ```{user_input}```

Sentiment:"""

message = [
    {
        "role": "user",
        "content": prompt.format(user_input=___)
    }
]

response = get_completion(message)
print(response)

---

## Section 6: Information Extraction

Extract structured information (names, entities, tags) from unstructured text.

### Q6.1 — Extract Model Names from a Paper Abstract

**Task:** Write a prompt that extracts AI/ML model names from a paper abstract and returns them as a JSON array `["model_name"]`. If no models found, return `["NA"]`.

In [ ]:
# TODO: Write the extraction prompt

prompt = """___  # Instruction: what to extract, what format, what to return if not found

Abstract: Large Language Models (LLMs), such as ChatGPT and GPT-4, have revolutionized natural language processing research and demonstrated potential in Artificial General Intelligence (AGI). However, the expensive training and deployment of LLMs present challenges to transparent and open academic research. To address these issues, this project open-sources the Chinese LLaMA and Alpaca...

Tags:"""

message = [{"role": "user", "content": prompt}]

response = get_completion(message)
print(response)

---

## Section 7: Translation & Tone Transformation

### Q7.1 — Machine Translation

**Task:** Write a prompt to translate `"Glad to be here!"` from English to Spanish.

In [ ]:
# TODO: Write the translation prompt

prompt = """___"""

message = [{"role": "user", "content": prompt}]

response = get_completion(message)
print(response)

### Q7.2 — Tone Transformation

LLMs can transform the **emotional tone** of text — not just the language.

**Task:** Write a prompt to transform `"Glad to be here!"` from a **happy** tone to a **super excited** tone.

In [ ]:
# TODO: Write the tone transformation prompt

prompt = """___"""

message = [{"role": "user", "content": prompt}]

response = get_completion(message)
print(response)

---

## Section 8: Role Playing

The model can take on different **personas** using the `system` message or by describing the role in the prompt.

### Q8.1 — Role Playing (Single Prompt)

**Task:** Write a single prompt that includes a conversation between a Human and an AI research assistant. The AI should have a technical/scientific tone. End with the human asking about black holes.

In [ ]:
# TODO: Write the role-playing prompt as a single text block

prompt = """___  # Describe the AI's persona and tone

Human: Hello, who are you?
AI: Greeting! I am an AI research assistant. How can I help you today?
Human: ___
AI:"""

message = [{"role": "user", "content": prompt}]

response = get_completion(message)
print(response)

### Q8.2 — Role Playing (System Message Approach)

**Task:** Achieve the **same result** as Q8.1 but using the proper `system` / `user` / `assistant` message structure. This is the **recommended** approach.

Fill in the roles and content for each message in the conversation.

In [ ]:
# TODO: Build the conversation using separate system/user/assistant messages

system_message = """___"""  # Persona description

messages = [
    {"role": "___", "content": system_message},             # System
    {"role": "___", "content": "___"},                      # User greeting
    {"role": "___", "content": "___"},                      # Assistant reply
    {"role": "___", "content": "___"},                      # User question about black holes
]

response = get_completion(messages)
print(response)

---

## Section 9: Code Generation

LLMs can generate code from natural language descriptions of database schemas and requirements.

### Q9.1 — Generate SQL from Schema Description

**Task:** Write a prompt that provides:
- Table `departments` with columns `[DepartmentId, DepartmentName]`
- Table `students` with columns `[DepartmentId, StudentId, StudentName]`
- Request: Create a MySQL query for all students in the Computer Science Department

In [ ]:
# TODO: Write the code generation prompt with schema + request

prompt = """___"""

message = [{"role": "user", "content": prompt}]

response = get_completion(message)
print(response)

---

## Section 10: Reasoning

LLMs can solve logical/math problems — but they do **much better** when asked to break problems into steps.

### Q10.1 — Step-by-Step Reasoning

**Task:** Write a prompt that asks whether the odd numbers in `[15, 32, 5, 13, 82, 7, 1]` add up to an even number. **Crucially**, instruct the model to solve step-by-step: first identify odd numbers, then add them, then check if the result is odd or even.

In [ ]:
# TODO: Write the reasoning prompt with step-by-step instructions

prompt = """___  # State the problem

___"""  # Instruction to break it into steps

message = [{"role": "user", "content": prompt}]

response = get_completion(message)
print(response)

---

## Section 11: Advanced — Few-Shot Prompting

**Few-shot prompting** provides the model with **examples** of input-output pairs before asking it to solve a new problem. This teaches the model the expected format and reasoning pattern.

### Q11.1 — Few-Shot Odd Number Problem

**Task:** Provide 4 examples of the odd-number-sum problem with answers (True/False), then ask the model to solve a 5th one.

🎯 **Hint:** The pattern is:
```
The odd numbers in this group add up to an even number: [numbers]
A: The answer is True/False.
```

In [ ]:
# TODO: Write a few-shot prompt with 4 examples + 1 question

prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: ___

The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: ___

The odd numbers in this group add up to an even number: 16, 11, 14, 4, 8, 13, 24.
A: ___

The odd numbers in this group add up to an even number: 17, 9, 10, 12, 13, 4, 2.
A: ___

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:"""

messages = [{"role": "user", "content": prompt}]

response = get_completion(messages)
print(response)

---

## Section 12: Advanced — Chain-of-Thought (CoT) Prompting

**Chain-of-Thought** prompting shows the model HOW to reason through a problem, not just the answer. The example includes intermediate reasoning steps.

### Q12.1 — Without CoT (Baseline)

**Task:** Ask the model directly if the odd numbers in `[15, 32, 5, 13, 82, 7, 1]` add up to an even number. Just ask for True/False — **no reasoning steps**.

In [ ]:
# TODO: Write a direct (no-CoT) prompt

prompt = """___"""

messages = [{"role": "user", "content": prompt}]

response = get_completion(messages)
print("Without CoT:", response)

### Q12.2 — With CoT (One Example with Reasoning)

**Task:** Now provide ONE example that includes the **reasoning steps** (identify odd numbers → add them → state result), then ask the model to solve the second problem the same way.

🎯 **Example pattern:**
```
The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.
```

In [ ]:
# TODO: Write a CoT prompt with one worked example

prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: ___  # Show the reasoning: which are odd, what's the sum, is it even?

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:"""

messages = [{"role": "user", "content": prompt}]

response = get_completion(messages)
print("With CoT:", response)

### Q12.3 — Zero-Shot CoT (Magic Phrase)

You can trigger chain-of-thought reasoning **without any examples** by appending a magic phrase!

**Task:** Write a word problem and append `"Let's think step by step."` at the end.

Problem: *"I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?"*

In [ ]:
# TODO: Write the problem and append the zero-shot CoT trigger phrase

prompt = """___  # The word problem

___"""  # The magic phrase that triggers step-by-step reasoning

messages = [{"role": "user", "content": prompt}]

response = get_completion(messages)
print(response)

---

## 📝 Self-Assessment Checklist

| # | Technique | Key Concept | Done? |
|---|-----------|------------|-------|
| 1 | Setup | `system`/`user`/`assistant` roles, `get_completion()` helper | ☐ |
| 2 | Basic Completion | Partial sentence → model completes it | ☐ |
| 3 | Summarization | Task-first, add constraints iteratively | ☐ |
| 4 | Question Answering | Context + Question + fallback instruction | ☐ |
| 5 | Text Classification | Delimit input with backticks, specify categories | ☐ |
| 6 | Information Extraction | Specify output format (JSON array) | ☐ |
| 7 | Translation & Tone | Language translation + emotional tone change | ☐ |
| 8 | Role Playing | Single-prompt vs system-message approach | ☐ |
| 9 | Code Generation | Schema description → SQL query | ☐ |
| 10 | Reasoning | Step-by-step instructions improve accuracy | ☐ |
| 11 | Few-Shot | Provide examples to teach the pattern | ☐ |
| 12 | Chain-of-Thought | Show reasoning steps; "Let's think step by step" | ☐ |